In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import lingualTF as la

import tensorflow as tf
#import keras

from keras.models import Sequential
from keras.layers import Convolution1D, AveragePooling1D, AveragePooling1D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization

[nltk_data] Downloading package wordnet to /Users/Seth/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     /Users/Seth/nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!


Using TensorFlow backend.


In [2]:
####################################
# REPLACE THESE WITH CORRECT PATHS #
####################################
docRanks = pd.read_csv('/Users/Seth/Documents/DSI/Capstone/DSI-Religion-2017/refData/docRanks.csv')
#docRanks.head()
docRanks = docRanks.ix[~((docRanks['groupName']=='YV03')|(docRanks['groupName']=='YV04'))]
docRanks.reset_index(inplace=True, drop=True)
docRanks.tail()

,groupName,rank
273,WBC347,2
274,WBC410,1
275,WBC418,1
276,WBC421,1
277,WBC422,1


In [3]:
def DSIacc(y_true, y_pred):
    return float(len([i for i in range(len(y_pred)) if abs(y_true[i][0]-y_pred[i][0])<=1])/float(len(y_pred)))

## POS TAGGINGS - skip a bit brother...

In [4]:
docsFolder = '/Users/Seth/Documents/DSI/Capstone/DSI-Religion-2017/data_dsicap_single/'
doc = docsFolder + docRanks['groupName'][0] + '/raw/' + docRanks['groupName'][0] + '.txt'
doc

'/Users/Seth/Documents/DSI/Capstone/DSI-Religion-2017/data_dsicap_single/ACLU01/raw/ACLU01.txt'

In [5]:
lo = la.lingualObject(doc)
print(lo.fileList)
fileName = lo.fileList[0]

['/Users/Seth/Documents/DSI/Capstone/DSI-Religion-2017/data_dsicap_single/ACLU01/raw/ACLU01.txt']


In [11]:
lo.pos_with_val()
lo.tags[fileName][:20]

['DT',
 'NNP',
 'NNP',
 'NNP',
 'VBZ',
 'VBG',
 'DT',
 'VBG',
 'JJ',
 'NN',
 'IN',
 'DT',
 'JJ',
 'VAL',
 'NNS',
 'IN',
 'NNP',
 'NN',
 'VAL',
 'IN']

In [7]:
keyInds = []
for i in range(0,len(lo.tokens[fileName])):
    if lo.tokens[fileName][i] in lo.keywords:
        keyInds += [i]
        
['VAL' if i in keyInds else lo.tags[fileName][i] for i in range(0,len(lo.tokens[fileName]))]

['DT',
 'NNP',
 'NNP',
 'NNP',
 'VBZ',
 'VBG',
 'DT',
 'VBG',
 'JJ',
 'NN',
 'IN',
 'DT',
 'JJ',
 'VAL',
 'NNS',
 'IN',
 'NNP',
 'NN',
 'VAL',
 'IN',
 'NNS',
 'WP',
 'RB',
 'VBD',
 'VBN',
 'IN',
 'NNP',
 'JJ',
 'JJ',
 'VAL',
 'CC',
 'IN',
 'NNP',
 'NNP',
 '.',
 'VAL',
 'NNS',
 ',',
 'NNP',
 'NNP',
 'CC',
 'NNP',
 'NNP',
 ',',
 'VBD',
 'JJ',
 'VAL',
 'IN',
 'DT',
 'NNP',
 'CC',
 'VBN',
 'CC',
 'VAL',
 'VB',
 'DT',
 'NN',
 'NN',
 '.',
 'PRP',
 'VBP',
 'RB',
 'VAL',
 'IN',
 'DT',
 'NNP',
 'NN',
 'IN',
 'NN',
 'VAL',
 'CD',
 'IN',
 'DT',
 'NNS',
 '.',
 'CD',
 'IN',
 'PRP',
 'VBD',
 'NN',
 'IN',
 'DT',
 'NNP',
 'VAL',
 'VBZ',
 'DT',
 'NN',
 'VAL',
 'VBN',
 'TO',
 'VAL',
 '.',
 'DT',
 'NNS',
 'VBN',
 'DT',
 'NN',
 'NN',
 'VAL',
 'NN',
 'NN',
 'VBG',
 'IN',
 'VBG',
 'NNS',
 'IN',
 'PRP',
 'VBD',
 'DT',
 'NN',
 'IN',
 'JJ',
 'NN',
 'MD',
 'VB',
 'TO',
 'JJ',
 'NN',
 '.',
 'IN',
 'DT',
 'NN',
 'VBD',
 'VBN',
 ',',
 'EX',
 'VBZ',
 'VBN',
 'DT',
 'JJ',
 'VAL',
 'IN',
 'PRP$',
 'NNS',
 'CC',
 'NN'

In [6]:
def docToNumpyPOS(doc,keywordCount=20, lengthThreshold=500, warnOnPad=False):
    vocab = la.allTags
    # tokenize, etc. document
    self = la.lingualObject(doc)
    fileName = self.fileList[0]
    # set the keywords
    self.pos_with_val(keywords = keywordCount, useVal=True)
    # create array of each word in doc with len(vocab)-dimensional sparse one-hot vectors
    sparse = [la.wordToInd(word, vocab, self.keywords) for word in self.tags[fileName]]
    # if less words than the threshold, pad with UNK
    if len(sparse) < lengthThreshold:
        if warnOnPad==True:
            print('PADDING: ' + doc + ' has only ' + str(len(sparse)) + ' words. Adding ' + str((lengthThreshold - len(sparse))) + ' UNKs.')
        sparse += [(len(vocab) - 2)] * (lengthThreshold - len(sparse))
    # create dense array of zeros, then fill in the correct ones
    docArr = np.zeros((lengthThreshold,len(vocab)), dtype=np.int) # default is 500, 1002
    for i in range(0, lengthThreshold):
        docArr[i][sparse[i]] = 1
    # 
    return docArr

In [7]:
docArr = docToNumpyPOS(doc)

In [17]:
docArr[19]

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [12]:
lo.pos_with_val()
lo.tags[fileName][:20]

['DT',
 'NNP',
 'NNP',
 'NNP',
 'VBZ',
 'VBG',
 'DT',
 'VBG',
 'JJ',
 'NN',
 'IN',
 'DT',
 'JJ',
 'VAL',
 'NNS',
 'IN',
 'NNP',
 'NN',
 'VAL',
 'IN']

## get data from file (processed in earlier scripts)

In [4]:
def docToNumpyPOS(doc,keywordCount=20, lengthThreshold=500, warnOnPad=False):
    # tokenize, etc. document
    self = la.lingualObject(doc)
    fileName = self.fileList[0]
    vocab = la.allTags
    # set the keywords
    self.pos_with_val(keywords = keywordCount, useVal=True)
    # create array of each word in doc with len(vocab)-dimensional sparse one-hot vectors
    sparse = [la.wordToInd(word, vocab, self.keywords) for word in self.tags[fileName]]
    # if less words than the threshold, pad with UNK
    if len(sparse) < lengthThreshold:
        if warnOnPad==True:
            print('PADDING: ' + doc + ' has only ' + str(len(sparse)) + ' words. Adding ' + str((lengthThreshold - len(sparse))) + ' UNKs.')
        sparse += [(len(vocab) - 2)] * (lengthThreshold - len(sparse))
    # create dense array of zeros, then fill in the correct ones
    docArr = np.zeros((lengthThreshold,len(vocab)), dtype=np.int) # default is 500, 1002
    for i in range(0, lengthThreshold):
        docArr[i][sparse[i]] = 1
    # 
    return docArr

In [5]:
# construct X matrix FROM SCRATCH
docsFolder = '/Users/Seth/Documents/DSI/Capstone/DSI-Religion-2017/data_dsicap_single/'
#
X = np.array([docToNumpyPOS(docsFolder + adoc + '/raw/' + adoc + '.txt') for adoc in tqdm(docRanks['groupName'])])
np.save('/Users/Seth/Documents/DSI/Capstone/big-data/X-single-POS.npy', X)
X.shape

100%|██████████| 278/278 [01:29<00:00,  1.60it/s]


(278, 500, 38)

In [6]:
# OR load X matrix from disk (already processed)
#X = np.load('/Users/Seth/Documents/DSI/Capstone/big-data/X-single-5k.npy')
X.shape

(278, 500, 38)

In [7]:
# construct Y matrix
Y = np.array([np.array([y]) for y in docRanks['rank']])
Y.shape

(278, 1)

In [8]:
# make a Y vector for classification
Y_cat = np.zeros((len(Y),9))

for i in range(0,len(Y)):
    Y_cat[i][Y[i]-1] = Y[i]

Y_cat.shape

(278, 9)

## shuffle the data into training and testing

In [9]:
# set the seed if you want to
np.random.seed(123)
#
shuf = docRanks.sample(frac=1).index
shuf

Int64Index([223,  20, 163,  29, 254, 237,  53, 174, 151, 156,
            ...
             96, 225, 214,  57, 123, 106,  83,  17, 230,  98],
           dtype='int64', length=278)

In [10]:
splitPoint = int(len(X) * .8)
#train = docRanks.iloc[shuf[:splitPoint]]
#test = docRanks.iloc[shuf[(splitPoint+1):]]
splitPoint

222

In [11]:
X_train = X[shuf[:splitPoint]]
Y_train = Y[shuf[:splitPoint]]
Y_cat_train = Y_cat[shuf[:splitPoint]]

X_test = X[shuf[(splitPoint+1):]]
Y_test = Y[shuf[(splitPoint+1):]]
Y_cat_test = Y_cat[shuf[(splitPoint+1):]]

## regression model

In [12]:
model = Sequential()
model.add(Convolution1D(64, 3, border_mode='same', input_shape=(X.shape[1],X.shape[2])))
model.add(BatchNormalization())
model.add(Activation('relu'))
#model.add(AveragePooling1D(pool_length=3))

model.add(Convolution1D(128, 7, border_mode='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling1D(pool_length=3))

model.add(Convolution1D(192, 3, border_mode='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling1D(pool_length=3))

model.add(Convolution1D(256, 3, border_mode='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling1D(pool_length=3))


In [13]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(4096, init='normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))

#model.add(Dense(4096, init='normal'))
model.add(Dense(1000, init='normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))

#model.add(Dense(1000, init='normal'))
model.add(Dense(200, init='normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))

In [14]:
#
#model.add(Dense(9, activation='softmax')) # for categorical_cross_entropy, below (classification)
model.add(Dense(1)) # for mse, below (regression)

In [15]:
def DSIacc(y_true, y_pred):
    return float(len([i for i in range(len(y_pred)) if abs(y_true[i][0]-y_pred[i][0])<=1])/float(len(y_pred)))

In [16]:
#model.compile(loss='categorical_cross_entropy', optimizer='rmsprop', metrics=['accuracy'])
model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['mean_absolute_error'])

### Batch size mattered a lot on this one
with `batch_size=1` we got huge numbers and 0 accuracy, with `batch_size=154` we got tiny numbers and 23% accuracy (basically all the 1's and maybe a few 2's). With `batch_size=32` we did a little better, but still not convincing...

In [17]:
#model.fit(X_train, Y_train, nb_epoch=10, batch_size=1)
#model.fit(X_train, Y_train, nb_epoch=10, batch_size= 154)
model.fit(X_train, Y_train, nb_epoch=10, batch_size=32) # 32 did way better than 1 on regression (opposite of classification)


Epoch 1/10
222/222 [==============================] - 10s - loss: 8.9617 - mean_absolute_error: 2.3292    
Epoch 2/10
222/222 [==============================] - 8s - loss: 4.3812 - mean_absolute_error: 1.6054     
Epoch 3/10
222/222 [==============================] - 8s - loss: 4.7966 - mean_absolute_error: 1.6206     
Epoch 4/10
222/222 [==============================] - 8s - loss: 3.5471 - mean_absolute_error: 1.4514     
Epoch 5/10
222/222 [==============================] - 9s - loss: 4.2213 - mean_absolute_error: 1.4901     
Epoch 6/10
222/222 [==============================] - 8s - loss: 3.0853 - mean_absolute_error: 1.3194     
Epoch 7/10
222/222 [==============================] - 6s - loss: 2.7038 - mean_absolute_error: 1.2665     
Epoch 8/10
222/222 [==============================] - 6s - loss: 2.3011 - mean_absolute_error: 1.0553     
Epoch 9/10
222/222 [==============================] - 7s - loss: 1.3865 - mean_absolute_error: 0.9274     
Epoch 10/10
222/222 [================

### evaluate

In [18]:
model.evaluate(X_test, Y_test, batch_size=1)

55/55 [==============================] - 1s     


[31.636980807666919, 3.1302032774144952]

In [19]:
y_pred = model.predict(X_test)
y_pred[:10]

array([[  2.65935779],
       [  1.7533114 ],
       [ 16.78481102],
       [  2.62229609],
       [ 23.63639259],
       [  2.28001046],
       [  2.55056596],
       [  2.42521453],
       [  2.13297009],
       [  1.97678781]], dtype=float32)

In [20]:
Y_test[:10]

array([[6],
       [2],
       [6],
       [5],
       [1],
       [8],
       [5],
       [2],
       [2],
       [1]])

In [21]:
DSIacc(Y_test,y_pred)

0.34545454545454546

## classification model

In [22]:
model = Sequential()
model.add(Convolution1D(64, 3, border_mode='same', input_shape=(X.shape[1],X.shape[2])))
model.add(BatchNormalization())
model.add(Activation('relu'))
#model.add(AveragePooling1D(pool_length=3))

model.add(Convolution1D(128, 7, border_mode='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling1D(pool_length=3))

model.add(Convolution1D(192, 3, border_mode='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling1D(pool_length=3))

model.add(Convolution1D(256, 3, border_mode='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling1D(pool_length=3))


In [23]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(4096, init='normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))

#model.add(Dense(4096, init='normal'))
model.add(Dense(1000, init='normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))

#model.add(Dense(1000, init='normal'))
model.add(Dense(200, init='normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))

In [24]:
#
model.add(Dense(9, activation='softmax')) # for categorical_cross_entropy, below (classification)
#model.add(Dense(1)) # for mse, below (regression)

In [25]:
#model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['mean_absolute_error', 'accuracy'])

In [26]:
#model.fit(data, labels, nb_epoch=10, batch_size=32) ### generic call from documentation
model.fit(X_train,Y_cat_train, nb_epoch=10, batch_size=32)
#model.fit(X_train, Y_cat_train, nb_epoch=10, batch_size= 1)

Epoch 1/10
222/222 [==============================] - 9s - loss: 1.8540 - mean_absolute_error: 0.5040 - acc: 0.0901     
Epoch 2/10
222/222 [==============================] - 7s - loss: 1.7172 - mean_absolute_error: 0.4740 - acc: 0.4054     
Epoch 3/10
222/222 [==============================] - 7s - loss: 1.5716 - mean_absolute_error: 0.4411 - acc: 0.5676     
Epoch 4/10
222/222 [==============================] - 6s - loss: 1.4568 - mean_absolute_error: 0.4139 - acc: 0.6847     
Epoch 5/10
222/222 [==============================] - 6s - loss: 1.3706 - mean_absolute_error: 0.3894 - acc: 0.7748     
Epoch 6/10
222/222 [==============================] - 6s - loss: 1.3246 - mean_absolute_error: 0.3702 - acc: 0.8604     
Epoch 7/10
222/222 [==============================] - 6s - loss: 1.3036 - mean_absolute_error: 0.3612 - acc: 0.8919     
Epoch 8/10
222/222 [==============================] - 6s - loss: 1.2670 - mean_absolute_error: 0.3459 - acc: 0.9279     
Epoch 9/10
222/222 [============

### evaluate

In [27]:
model.evaluate(X_test, Y_cat_test, batch_size=1)

54/55 [============================>.] - ETA: 0s

[1.8108420699834824, 0.49770034822550685, 0.14545454545454545]

In [28]:
y_pred = model.predict(X_test)
y_pred_class = [[(np.argmax(pred)+1)] for pred in y_pred]
y_pred_class[:10]

[[2], [2], [2], [2], [2], [2], [2], [2], [2], [2]]

In [29]:
Y_test[:10]

array([[6],
       [2],
       [6],
       [5],
       [1],
       [8],
       [5],
       [2],
       [2],
       [1]])

In [30]:
DSIacc(Y_test,y_pred_class)

0.41818181818181815

# DROPOUT

## regression model - WITH DROPOUT

In [31]:
model = Sequential()
model.add(Convolution1D(64, 3, border_mode='same', input_shape=(X.shape[1],X.shape[2])))
model.add(BatchNormalization())
model.add(Activation('relu'))
#model.add(AveragePooling1D(pool_length=3))

model.add(Convolution1D(128, 7, border_mode='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling1D(pool_length=3))

model.add(Convolution1D(192, 3, border_mode='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling1D(pool_length=3))

model.add(Convolution1D(256, 3, border_mode='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling1D(pool_length=3))


In [32]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(4096, init='normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dropout(.5))

#model.add(Dense(4096, init='normal'))
model.add(Dense(1000, init='normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dropout(.25))

#model.add(Dense(1000, init='normal'))
model.add(Dense(200, init='normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))

In [33]:
#
model.add(Dense(1)) # for mse, below (regression)

In [34]:
#model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['mean_absolute_error', 'accuracy'])

In [35]:
#model.fit(X_train, Y_train, nb_epoch=10, batch_size= 1) #just like without dropout, this produced crazy predictions
model.fit(X_train,Y_train, nb_epoch=10, batch_size=32)

Epoch 1/10
222/222 [==============================] - 11s - loss: 8.8932 - mean_absolute_error: 2.3117 - acc: 0.1351    
Epoch 2/10
222/222 [==============================] - 8s - loss: 3.6514 - mean_absolute_error: 1.5562 - acc: 0.1802     
Epoch 3/10
222/222 [==============================] - 8s - loss: 3.9243 - mean_absolute_error: 1.6134 - acc: 0.2027     
Epoch 4/10
222/222 [==============================] - 7s - loss: 3.3670 - mean_absolute_error: 1.4861 - acc: 0.1937     
Epoch 5/10
222/222 [==============================] - 7s - loss: 3.1560 - mean_absolute_error: 1.4248 - acc: 0.2027     
Epoch 6/10
222/222 [==============================] - 7s - loss: 2.6895 - mean_absolute_error: 1.3138 - acc: 0.2477     
Epoch 7/10
222/222 [==============================] - 7s - loss: 2.8525 - mean_absolute_error: 1.3155 - acc: 0.2432     
Epoch 8/10
222/222 [==============================] - 7s - loss: 2.6366 - mean_absolute_error: 1.3213 - acc: 0.2162     
Epoch 9/10
222/222 [============

### evaluate

In [36]:
model.evaluate(X_test, Y_test, batch_size=1)

53/55 [===========================>..] - ETA: 0s

[7.2526552242396232, 2.1935987277464433, 0.10909090909090909]

In [37]:
y_pred = model.predict(X_test)
y_pred[:10]

array([[ 2.21051598],
       [ 0.39288539],
       [ 1.45857155],
       [ 1.41592383],
       [ 1.87833953],
       [ 1.77807879],
       [ 1.76397121],
       [ 1.67770183],
       [ 1.86329281],
       [ 1.75807273]], dtype=float32)

In [38]:
Y_test[:10]

array([[6],
       [2],
       [6],
       [5],
       [1],
       [8],
       [5],
       [2],
       [2],
       [1]])

In [39]:
DSIacc(Y_test,y_pred)

0.2909090909090909

## classification model - WITH DROPOUT

In [40]:
model = Sequential()
model.add(Convolution1D(64, 3, border_mode='same', input_shape=(X.shape[1],X.shape[2])))
model.add(BatchNormalization())
model.add(Activation('relu'))
#model.add(AveragePooling1D(pool_length=3))

model.add(Convolution1D(128, 7, border_mode='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling1D(pool_length=3))

model.add(Convolution1D(192, 3, border_mode='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling1D(pool_length=3))

model.add(Convolution1D(256, 3, border_mode='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(AveragePooling1D(pool_length=3))


In [41]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(4096, init='normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dropout(.5))

#model.add(Dense(4096, init='normal'))
model.add(Dense(1000, init='normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Dropout(.25))

#model.add(Dense(1000, init='normal'))
model.add(Dense(200, init='normal'))
model.add(BatchNormalization())
model.add(Activation('relu'))

In [42]:
#
model.add(Dense(9, activation='softmax')) # for categorical_cross_entropy, below (classification)
#model.add(Dense(1)) # for mse, below (regression)

In [43]:
#model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['mean_absolute_error', 'accuracy'])

In [44]:
#model.fit(data, labels, nb_epoch=10, batch_size=32) ### generic call from documentation
#model.fit(X_train,Y_cat_train, nb_epoch=10, batch_size=32)
model.fit(X_train, Y_cat_train, nb_epoch=10, batch_size= 1)

Epoch 1/10
222/222 [==============================] - 52s - loss: 1.8437 - mean_absolute_error: 0.5022 - acc: 0.2928    
Epoch 2/10
222/222 [==============================] - 49s - loss: 1.8423 - mean_absolute_error: 0.5019 - acc: 0.3018    
Epoch 3/10
222/222 [==============================] - 49s - loss: 1.8409 - mean_absolute_error: 0.5016 - acc: 0.3018    
Epoch 4/10
222/222 [==============================] - 49s - loss: 1.8395 - mean_absolute_error: 0.5013 - acc: 0.3018    
Epoch 5/10
222/222 [==============================] - 50s - loss: 1.8379 - mean_absolute_error: 0.5009 - acc: 0.3018    
Epoch 6/10
222/222 [==============================] - 49s - loss: 1.8363 - mean_absolute_error: 0.5005 - acc: 0.3018    
Epoch 7/10
222/222 [==============================] - 50s - loss: 1.8347 - mean_absolute_error: 0.5002 - acc: 0.3018    
Epoch 8/10
222/222 [==============================] - 50s - loss: 1.8330 - mean_absolute_error: 0.4997 - acc: 0.3018    
Epoch 9/10
222/222 [============

### evaluate

In [45]:
model.evaluate(X_test, Y_cat_test, batch_size=1)

54/55 [============================>.] - ETA: 0s

[2.0020201897079293, 0.52323233837431127, 0.0]

In [46]:
y_pred = model.predict(X_test)
y_pred_class = [[(np.argmax(pred)+1)] for pred in y_pred]
y_pred_class[:10]

[[7], [7], [7], [7], [7], [7], [7], [6], [7], [7]]

In [47]:
Y_test[:10]

array([[6],
       [2],
       [6],
       [5],
       [1],
       [8],
       [5],
       [2],
       [2],
       [1]])

In [48]:
DSIacc(Y_test,y_pred_class)

0.10909090909090909

## with a larger batch size (spoiler, it does really badly)

In [56]:
#THE FULL SETUP, compacted
model = Sequential(); model.add(Convolution1D(64, 3, border_mode='same', input_shape=(X.shape[1],X.shape[2]))); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(AveragePooling1D(pool_length=3)); model.add(Convolution1D(128, 7, border_mode='same')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(AveragePooling1D(pool_length=3)); model.add(Convolution1D(192, 3, border_mode='same')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(AveragePooling1D(pool_length=3)); model.add(Convolution1D(256, 3, border_mode='same')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(AveragePooling1D(pool_length=3))
model.add(Flatten()); model.add(Dense(4096, init='normal')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(Dropout(.5)); model.add(Dense(1000, init='normal')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(Dropout(.25)); model.add(Dense(200, init='normal')); model.add(BatchNormalization()); model.add(Activation('relu'))
#
model.add(Dense(9, activation='softmax')) # for categorical_cross_entropy, below (classification)
#model.add(Dense(1)) # for mse, below (regression)


In [57]:
model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['mean_absolute_error', 'accuracy'])

In [58]:
model.fit(X_train,Y_cat_train, nb_epoch=10, batch_size=154)

Epoch 1/10
222/222 [==============================] - 7s - loss: 1.8719 - mean_absolute_error: 0.5051 - acc: 0.0405     
Epoch 2/10
222/222 [==============================] - 3s - loss: 1.8505 - mean_absolute_error: 0.5000 - acc: 0.1261     
Epoch 3/10
222/222 [==============================] - 2s - loss: 1.8365 - mean_absolute_error: 0.4972 - acc: 0.1622     
Epoch 4/10
222/222 [==============================] - 2s - loss: 1.8325 - mean_absolute_error: 0.4966 - acc: 0.1622     
Epoch 5/10
222/222 [==============================] - 2s - loss: 1.8082 - mean_absolute_error: 0.4902 - acc: 0.2297     
Epoch 6/10
222/222 [==============================] - 3s - loss: 1.7914 - mean_absolute_error: 0.4869 - acc: 0.3153     
Epoch 7/10
222/222 [==============================] - 3s - loss: 1.7974 - mean_absolute_error: 0.4886 - acc: 0.2973     
Epoch 8/10
222/222 [==============================] - 2s - loss: 1.7752 - mean_absolute_error: 0.4822 - acc: 0.3423     
Epoch 9/10
222/222 [============

### evaluate

In [59]:
model.evaluate(X_test, Y_cat_test, batch_size=1)

54/55 [============================>.] - ETA: 0s

[1.8173169836401939, 0.4995492853901603, 0.0]

In [60]:
y_pred = model.predict(X_test)
y_pred_class = [[(np.argmax(pred)+1)] for pred in y_pred]
y_pred_class[:10]

[[7], [7], [7], [7], [7], [7], [7], [7], [7], [7]]

In [61]:
Y_test[:10]

array([[6],
       [2],
       [6],
       [5],
       [1],
       [8],
       [5],
       [2],
       [2],
       [1]])

In [62]:
DSIacc(Y_test,y_pred_class)

0.10909090909090909

### NOTE: this looks a little encouraging, but it's not
It just guesses 3 for everything, and since about half the examples are 2,3,4, it counts those are correct. Also, if you run this identical thing a few times, it will come out differently, but always picking the same number for every testing observation. Last time it pick 7 for everything and got 10% accuracy.

I also tried batch size 32 and got about the same result.


## running for more epochs (back to batch size 1)

In [63]:
#THE FULL SETUP, compacted
model = Sequential(); model.add(Convolution1D(64, 3, border_mode='same', input_shape=(X.shape[1],X.shape[2]))); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(AveragePooling1D(pool_length=3)); model.add(Convolution1D(128, 7, border_mode='same')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(AveragePooling1D(pool_length=3)); model.add(Convolution1D(192, 3, border_mode='same')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(AveragePooling1D(pool_length=3)); model.add(Convolution1D(256, 3, border_mode='same')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(AveragePooling1D(pool_length=3))
model.add(Flatten()); model.add(Dense(4096, init='normal')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(Dropout(.5)); model.add(Dense(1000, init='normal')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(Dropout(.25)); model.add(Dense(200, init='normal')); model.add(BatchNormalization()); model.add(Activation('relu'))
#
model.add(Dense(9, activation='softmax')) # for categorical_cross_entropy, below (classification)
#model.add(Dense(1)) # for mse, below (regression)


In [64]:
#model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['mean_absolute_error', 'accuracy'])

In [65]:
model.fit(X_train, Y_cat_train, nb_epoch=20, batch_size= 1)

Epoch 1/20
222/222 [==============================] - 28s - loss: 1.8437 - mean_absolute_error: 0.5022 - acc: 0.2793    
Epoch 2/20
222/222 [==============================] - 25s - loss: 1.8423 - mean_absolute_error: 0.5019 - acc: 0.3018    
Epoch 3/20
222/222 [==============================] - 24s - loss: 1.8409 - mean_absolute_error: 0.5016 - acc: 0.3018    
Epoch 4/20
222/222 [==============================] - 23s - loss: 1.8395 - mean_absolute_error: 0.5013 - acc: 0.3018    
Epoch 5/20
222/222 [==============================] - 23s - loss: 1.8379 - mean_absolute_error: 0.5009 - acc: 0.3018    
Epoch 6/20
222/222 [==============================] - 22s - loss: 1.8363 - mean_absolute_error: 0.5006 - acc: 0.3018    
Epoch 7/20
222/222 [==============================] - 22s - loss: 1.8347 - mean_absolute_error: 0.5002 - acc: 0.3018    
Epoch 8/20
222/222 [==============================] - 22s - loss: 1.8330 - mean_absolute_error: 0.4998 - acc: 0.3018    
Epoch 9/20
222/222 [============

### evaluate

In [66]:
model.evaluate(X_test, Y_cat_test, batch_size=1)

51/55 [==========================>...] - ETA: 0s

[1.8121212101795456, 0.48686870052055881, 0.16363636363636364]

In [67]:
y_pred = model.predict(X_test)
y_pred_class = [[(np.argmax(pred)+1)] for pred in y_pred]
y_pred_class[:10]

[[8], [7], [8], [5], [8], [8], [8], [5], [5], [7]]

In [68]:
Y_test[:10]

array([[6],
       [2],
       [6],
       [5],
       [1],
       [8],
       [5],
       [2],
       [2],
       [1]])

In [69]:
DSIacc(Y_test,y_pred_class)

0.2909090909090909

In [74]:
# this is weird, right?
y_pred_class_low = [[(np.argmax(pred)-2)] for pred in y_pred]
y_pred_class_low[:10]
DSIacc(Y_test,y_pred_class_low) 

0.45454545454545453

## with crossentropy instead

In [76]:
#THE FULL SETUP, compacted
model = Sequential(); model.add(Convolution1D(64, 3, border_mode='same', input_shape=(X.shape[1],X.shape[2]))); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(AveragePooling1D(pool_length=3)); model.add(Convolution1D(128, 7, border_mode='same')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(AveragePooling1D(pool_length=3)); model.add(Convolution1D(192, 3, border_mode='same')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(AveragePooling1D(pool_length=3)); model.add(Convolution1D(256, 3, border_mode='same')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(AveragePooling1D(pool_length=3))
model.add(Flatten()); model.add(Dense(4096, init='normal')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(Dropout(.5)); model.add(Dense(1000, init='normal')); model.add(BatchNormalization()); model.add(Activation('relu')); model.add(Dropout(.25)); model.add(Dense(200, init='normal')); model.add(BatchNormalization()); model.add(Activation('relu'))
#
model.add(Dense(9, activation='softmax')) # for categorical_cross_entropy, below (classification)
#model.add(Dense(1)) # for mse, below (regression)


In [77]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['mean_absolute_error','accuracy'])
#model.compile(loss='mean_squared_error', optimizer='sgd', metrics=['mean_absolute_error', 'accuracy'])

In [78]:
model.fit(X_train, Y_cat_train, nb_epoch=20, batch_size= 1)

Epoch 1/20
222/222 [==============================] - 43s - loss: 8.1251 - mean_absolute_error: 0.5017 - acc: 0.2973    
Epoch 2/20
222/222 [==============================] - 41s - loss: 7.9460 - mean_absolute_error: 0.5006 - acc: 0.3018    
Epoch 3/20
222/222 [==============================] - 41s - loss: 7.7965 - mean_absolute_error: 0.4996 - acc: 0.3018    
Epoch 4/20
222/222 [==============================] - 41s - loss: 7.6692 - mean_absolute_error: 0.4986 - acc: 0.3018    
Epoch 5/20
222/222 [==============================] - 41s - loss: 7.5572 - mean_absolute_error: 0.4976 - acc: 0.3018    
Epoch 6/20
222/222 [==============================] - 41s - loss: 7.4666 - mean_absolute_error: 0.4967 - acc: 0.3018    
Epoch 7/20
222/222 [==============================] - 41s - loss: 7.3891 - mean_absolute_error: 0.4958 - acc: 0.3018    
Epoch 8/20
222/222 [==============================] - 41s - loss: 7.3280 - mean_absolute_error: 0.4949 - acc: 0.3018    
Epoch 9/20
222/222 [============

### evaluate

In [79]:
model.evaluate(X_test, Y_cat_test, batch_size=1)

53/55 [===========================>..] - ETA: 0s

[51.870961750637399, 0.50303031788630914, 0.090909090909090912]

In [80]:
y_pred = model.predict(X_test)
y_pred_class = [[(np.argmax(pred)+1)] for pred in y_pred]
y_pred_class[:10]

[[7], [3], [8], [8], [5], [8], [3], [4], [8], [2]]

In [81]:
Y_test[:10]

array([[6],
       [2],
       [6],
       [5],
       [1],
       [8],
       [5],
       [2],
       [2],
       [1]])

In [82]:
DSIacc(Y_test,y_pred_class)

0.32727272727272727